In [2]:
# nltk 사용시 이 명령어 써줘야 함
import nltk

# 가상 환경 내에 설치될 경로 지정
nltk.data.path.append('/venv/nltk_data')

# 체크
print(nltk.data.path)

['C:\\Users\\rrhhk/nltk_data', 'C:\\Users\\rrhhk\\Desktop\\Codes\\Daegwanbun2\\venv\\nltk_data', 'C:\\Users\\rrhhk\\Desktop\\Codes\\Daegwanbun2\\venv\\share\\nltk_data', 'C:\\Users\\rrhhk\\Desktop\\Codes\\Daegwanbun2\\venv\\lib\\nltk_data', 'C:\\Users\\rrhhk\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data', '/venv/nltk_data']


In [3]:
from sklearn.datasets import fetch_20newsgroups

categories = ["alt.atheism", "soc.religion.christian", "comp.graphics", "sci.med"]

twenty_train = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [4]:
print(twenty_train.data[0])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



In [5]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [7]:
count_vect_s = CountVectorizer(stop_words="english")
X_train_counts_s = count_vect_s.fit_transform(twenty_train.data)
X_train_counts_s.shape

(2257, 35482)

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tdidf_transformer = TfidfTransformer()
X_train_tfidf = tdidf_transformer.fit_transform(X_train_counts.toarray())
X_train_tfidf.shape

(2257, 35788)

In [9]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

docs_new = ["cancer patient", "OpenGL on the GPU is fast"]
X_new_counts = count_vectorizer.transform(docs_new)
X_new_tdidf = tdidf_transformer.transform(X_new_counts)

predicted = nb.predict(X_new_tdidf)

for doc, category in zip(docs_new, predicted):
    print(f"{doc} => {twenty_train.target_names[category]}")

cancer patient => sci.med
OpenGL on the GPU is fast => comp.graphics


In [19]:
from sklearn.datasets import load_files
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

categories = ['crime', 'technology', 'politics', 'science', 'entertainment','space']

train_data = load_files(
    container_path='text/train',
    categories=categories,
    shuffle=True,
    encoding='utf-8',
    decode_error='replace',
    random_state=42
)
X_train, y_train = train_data.data, train_data.target

test_data = load_files(
    container_path='text/test',
    categories=categories,
    shuffle=True,
    encoding='utf-8',
    decode_error='replace',
    random_state=42
)
X_test, y_test = test_data.data, test_data.target

In [33]:
from sklearn.naive_bayes import MultinomialNB

# Define the pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 3))),  # Converts text to BoW
    ('tfidf', TfidfTransformer()),     # Converts BoW to TF-IDF
    ('classifier', MultinomialNB())
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate on the test set
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=train_data.target_names))

               precision    recall  f1-score   support

        crime       0.72      0.99      0.83        89
entertainment       1.00      0.08      0.15        61
     politics       0.45      0.97      0.62        87
      science       1.00      0.43      0.60        65
        space       0.93      0.98      0.96        65
   technology       0.93      0.24      0.38        59

     accuracy                           0.66       426
    macro avg       0.84      0.61      0.59       426
 weighted avg       0.81      0.66      0.61       426



In [36]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Load dataset using load_files
dataset = load_files(
    container_path='text/train',
    categories=categories,
    shuffle=True,
    encoding='utf-8',
    decode_error='replace',
    random_state=42
)
X, y = dataset.data, dataset.target

# Function to analyze dataset
def analyze_text_data(data):
    # Total documents
    num_docs = len(data)

    # Total word count
    total_words = sum(len(doc.split()) for doc in data)

    # Vocabulary size using CountVectorizer
    vectorizer = CountVectorizer()
    X_vec = vectorizer.fit_transform(data)
    vocab_size = len(vectorizer.get_feature_names_out())

    # Type-to-Token Ratio (TTR)
    unique_words = set(word for doc in data for word in doc.split())
    ttr = len(unique_words) / total_words

    # Display results
    print(f"Number of Documents: {num_docs}")
    print(f"Total Words: {total_words}")
    print(f"Vocabulary Size: {vocab_size}")
    print(f"Type-to-Token Ratio (TTR): {ttr:.3f}")

    # Optional: Top 10 frequent words
    word_counts = Counter(word for doc in data for word in doc.split())
    print("Top 10 Frequent Words:", word_counts.most_common(10))

# Analyze the dataset
analyze_text_data(X)

Number of Documents: 1722
Total Words: 1261483
Vocabulary Size: 36404
Type-to-Token Ratio (TTR): 0.069
Top 10 Frequent Words: [('the', 63417), ('to', 35838), ('and', 28691), ('of', 28667), ('a', 27034), ('in', 23691), ('that', 15370), ('for', 10610), ('was', 10558), ('on', 10429)]


In [34]:
import matplotlib.pyplot as plt

# Train the pipeline (from the earlier example)
pipeline.fit(X_train, y_train)

# Extract the trained DecisionTreeClassifier
classifier = pipeline.named_steps['classifier']

# Extract feature names from the vectorizer
feature_names = pipeline.named_steps['vectorizer'].get_feature_names_out()

# Get feature importances from the DecisionTreeClassifier
importances = classifier.feature_importances_

# Pair feature names with their importances
feature_importances = list(zip(feature_names, importances))

# Sort features by importance
sorted_features = sorted(feature_importances, key=lambda x: x[1], reverse=True)

# Get the top 20 features
top_20_features = sorted_features[:20]

# Prepare data for visualization
top_20_names = [feature[0] for feature in top_20_features]
top_20_values = [feature[1] for feature in top_20_features]

# Plot
plt.figure(figsize=(10, 6))
plt.barh(top_20_names, top_20_values)
plt.xlabel("Feature Importance")
plt.ylabel("Feature Name")
plt.title("Top 20 Feature Importances in Decision Tree")
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

AttributeError: 'MultinomialNB' object has no attribute 'feature_importances_'

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# Define pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB())
])

# Define parameter grid
param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'vectorizer__max_df': [0.75, 1.0],
    'vectorizer__min_df': [1, 2],
    'classifier__alpha': [0.1, 1, 10]
}

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters: {'classifier__alpha': 0.1, 'vectorizer__max_df': 0.75, 'vectorizer__min_df': 2, 'vectorizer__ngram_range': (1, 1)}
Best Accuracy: 0.9111498257839722


In [40]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', SVC())
])

# Define parameter grid
param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'vectorizer__max_df': [0.75, 1.0],
    'vectorizer__min_df': [1, 2]
}

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Parameters: {'vectorizer__max_df': 0.75, 'vectorizer__min_df': 2, 'vectorizer__ngram_range': (1, 1)}
Best Accuracy: 0.9076655052264808
